In [ ]:
#import nécessaire
import matplotlib.pyplot as plt
from gtts import gTTS
from pygame import mixer
import os
import time
import numpy as np
from tempfile import TemporaryFile
import speech_recognition as sr
from googletrans import Translator
from watson_developer_cloud import SpeechToTextV1
from ibm_watson import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions

#listes des langues
langueparlee=['en-US','fr-FR','es-ES']


#phrase réponse enregistrée
joie="Tu as l'air bien aujourd'hui"
colere="Calme toi, ce n'est pas grave, essaye de penser à autre chose"
peur="Ne t'inquiète pas, tout va bien se passer"
tristesse="Essaye de penser à autre chose, ça va aller"
degout="Passons vite à la question suivante"
defaut="ok, on passe à la question suivante"
erreur="Veux-tu que je répète la question oui ou non"
oui="Très bien, je répète la question"
debut="Hello, how are you"


#définition des variables globales
YES="Yes"
NO="No"
validation=0
langueactuelle='fr-FR'
listquestions=list()
nombrequestions=list()
posneg=list()
joy=list()
anger=list()
disgust=list()
sadness=list()
fear=list()



#définiton de la langue parlée
def langueparler():
    IBM_URL = 'https://gateway-lon.watsonplatform.net/speech-to-text/api'
    IBM_API = "-3kCDJhK9MLpl6G_0jZOXo7WsDwkQW8ALqTkTm6WB5Wu"
    service = SpeechToTextV1(url=IBM_URL,iam_apikey=IBM_API)
    langues=['anglais','fr-FR_BroadbandModel','es-ES_BroadbandModel']
    i=0
    texttradaudio0 = gTTS(text=debut, lang='en-US', slow=False)
    mixer.init()
    sf0 = TemporaryFile()
    texttradaudio0.write_to_fp(sf0)
    sf0.seek(0)
    mixer.music.load(sf0)
    mixer.music.play()
    time.sleep(5)
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Dites queslques chose")
        audiofinal0 = r.listen(source)
        with open(join(dirname('recording_0.wav'), "/Users/camil/TESTPROJETMONS/recording_0.wav"), 'rb') as audio_file:
            textlanguage=service.recognize(audiofinal0, content_type='audio/wav').get_result()
            conf=text['results']
            conf3=conf[0]
            conf4=conf3['alternatives']
            conf5=conf4[0]
            conf6=conf5['confidence']
            if conf6<0.5:
                i=1
            while conf6<0.5:
                testlang=langues[i]
                textlanguage=service.recognize(audiofinal0, content_type='audio/wav', model=testlang).get_result()
                conf=text['results']
                conf3=conf[0]
                conf4=conf3['alternatives']
                conf5=conf4[0]
                conf6=conf5['confidence']
                i=i+1
            else:
                langueactuelle=langues[i]
                
                

#récupération des questions
def questions(namefolder):
    fichier=open(namefolder,'r',encoding = 'unicode_escape') 
    text=fichier.read()
    numquest=1
    for interro in text.split('?'):
        interro = interro.replace(u'\xa0', u' ')
        interro = interro.replace(u'\n', u' ')
        listquestions.append(interro)
        nombrequestions.append(numquest)
        numquest=numquest+1
    fichier.close()
    nombrequestions.remove(len(nombrequestions))
    del listquestions[len(nombrequestions)]
    return listquestions,nombrequestions


#calcul de la moyenne et création de graphique
def moyennelistes(posneg,joy,anger,disgust,sadness,fear):
    #création des variables moyennes
    resposneg=''
    mposneg=0
    mjoy=0
    manger=0
    mdisgust=0
    msadness=0
    mfear=0
    #calcul des moyennes
    for element in posneg:
        mposneg=mposneg+element
    mposneg=mposneg/len(posneg)
    if mposneg<0:
        resposneg='Aperçu de la vie négatif'
    elif mposneg>0:
        resposneg='Aperçu de la vie positif'
    else:
        resposneg='Neutre'
    for element in joy:
        mjoy=mjoy+element
    mjoy=mjoy/len(joy)
    for element in anger:
        manger=manger+element
    manger=manger/len(anger)
    for element in disgust:
        mdisgust=mdisgust+element
    mdisgust=mdisgust/len(disgust)
    for element in sadness:
        msadness=msadness+element
    msadness=msadness/len(sadness)
    for element in fear:
        mfear=mfear+element
    mfear=mfear/len(fear)
    #création des graphiques
    #graphique en fonction des résultats
    plt.figure(1)
    plt.title("Résultats du test")
    vnombrequest=np.array(nombrequestions)
    vjoy=np.array(joy)
    vanger=np.array(anger)
    vdisgust=np.array(disgust)
    vsadness=np.array(sadness)
    vfear=np.array(fear)
    plt.plot(vnombrequest,vjoy,'g',label="Joie")
    plt.plot(vnombrequest,vanger,'r',label="Colère")
    plt.plot(vnombrequest,vdisgust,'c',label="Dégoût")
    plt.plot(vnombrequest,vsadness,'m',label="Tristesse")
    plt.plot(vnombrequest,vfear,'y',label="Peur")
    plt.axis([0, len(nombrequestions), 0, 1])
    plt.xlabel('Numéro des questions')
    plt.ylabel('Résultats')
    plt.legend()
    plt.show()
    #graphique en fonction des moyennes
    plt.figure(2)
    plt.title("Résultats du test émotion")
    labels = 'Joie', 'Colère', 'Dégoût', 'Tristesse', 'Peur'
    plt.axis("equal")
    plt.pie([mjoy, manger, mdisgust, msadness, mfear],labels=labels,autopct="%1.1f")  
    plt.show()
    #affichage positif ou negatif 
    print(resposneg)
    texttradaudio2 = gTTS(text=resposneg, lang=langueactuelle, slow=False)
    mixer.init()
    sf2 = TemporaryFile()
    texttradaudio2.write_to_fp(sf2)
    sf2.seek(0)
    mixer.music.load(sf2)
    mixer.music.play()


#application principale
langueparler()
time.sleep(3)
questions("C:/Users/camil/Desktop/Projetmonsprojetbase/questionslambdas.txt")
for ask in listquestions:
    texttradaudio = gTTS(text=ask, lang=langueactuelle, slow=False)
    mixer.init()
    sf = TemporaryFile()
    texttradaudio.write_to_fp(sf)
    sf.seek(0)
    mixer.music.load(sf)
    mixer.music.play()
    print(ask)    
    #enregistrement réponse 
    r = sr.Recognizer()
    translator = Translator()
    with sr.Microphone() as source:
        print("Dites queslques chose")
        audiofinal = r.listen(source)
    try:
        text = r.recognize_google(audiofinal,language=langueactuelle)
    except sr.UnknownValueError:
        validation=1
    while validation==1:
        erreurtrans = translator.translate(erreur,dest=langueactuelle)
        erreurtrans2 = erreurtrans.text
        texttradaudio4 = gTTS(text=erreurtrans2, lang=langueactuelle, slow=False)
        mixer.init()
        sf4 = TemporaryFile()
        texttradaudio4.write_to_fp(sf4)
        sf4.seek(0)
        mixer.music.load(sf4)
        mixer.music.play()
        with sr.Microphone() as source:
            print("Dites queslques chose")
            audiofinalrep = r.listen(source)
        textrepcourte = r.recognize_google(audiofinalrep,language=langueactuelle)
        textrepcourtetrad = translator.translate(textrepcourte)
        textrepcourtetrad2=textrepcourtetrad.text
        print(textrepcourtetrad2)
        if textrepcourtetrad2==YES:
            ouitrans = translator.translate(oui,dest=langueactuelle)
            ouitrans2=ouitrans.text
            texttradaudio6 = gTTS(text=ouitrans2, lang=langueactuelle, slow=False)
            mixer.init()
            sf6 = TemporaryFile()
            texttradaudio6.write_to_fp(sf6)
            sf6.seek(0)
            mixer.music.load(sf6)
            mixer.music.play()
            time.sleep(4)
            mixer.init()
            sf5 = TemporaryFile()
            texttradaudio.write_to_fp(sf5)
            sf5.seek(0)
            mixer.music.load(sf5)
            mixer.music.play()
            time.sleep(4)
            validation=0
        else:
            validation=0
            break
            break
    translations = translator.translate(text)
    KEY_IBM = "74v192fabJsRIZYIahiYu15mrzTZ1F0ISEA9hCGIKU5t"
    URL_IBM="https://gateway-wdc.watsonplatform.net/natural-language-understanding/api"
    texttrad = translations.text
    alchemy_language = NaturalLanguageUnderstandingV1(version='2018-11-16',iam_apikey=KEY_IBM,url=URL_IBM)
    sentiment = alchemy_language.analyze(text=texttrad,features=Features(emotion=EmotionOptions(),sentiment=SentimentOptions())).get_result()
    #tri des données
    emo = sentiment['emotion']
    emo2 = emo['document']
    emo3 = emo2['emotion']
    #trouver l'émotion principale de la réponse
    emotionrepvaleur=list(emo3.keys())[list(emo3.values()).index(max(emo3.values()))]
    if emotionrepvaleur=='joy':
        repafaire=joie
    elif emotionrepvaleur=='anger':
        repafaire=colere
    elif emotionrepvaleur=='sadness':
        repafaire=tristesse
    elif emotionrepvaleur=='fear':
        repafaire=peur
    elif emotionrepvaleur=='disgust':
        repafaire=degout
    else:
        repafaire=defaut
    repafairetrans = translator.translate(repafaire,dest=langueactuelle)
    repafairetrans2=repafairetrans.text
    texttradaudio3 = gTTS(text=repafairetrans2, lang=langueactuelle, slow=False)
    mixer.init()
    sf3 = TemporaryFile()
    texttradaudio3.write_to_fp(sf3)
    sf3.seek(0)
    mixer.music.load(sf3)
    mixer.music.play()
    time.sleep(4)
    #remplissage des listes des résultats
    joy.append(emo3['joy'])
    anger.append(emo3['anger'])
    disgust.append(emo3['disgust'])
    sadness.append(emo3['sadness'])
    fear.append(emo3['fear'])
    sent = sentiment['sentiment']
    sent2 = sent['document']
    sent3 = sent2['score']
    posneg.append(sent3)
moyennelistes(posneg,joy,anger,disgust,sadness,fear)   

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Dites queslques chose
